In [2]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load

In [8]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

# library to see the progress during the training
from tqdm.notebook import tqdm
tqdm().pandas()

0it [00:00, ?it/s]

In [14]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# get all images and your respective captions
def all_img_captions(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions = {}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

def cleaning_text(captions):
    table = str.maketrans('', '', string.punctuation)
    for img, caps in captions.items():
        for i, img_caption in enumerate(caps):
            
            img_caption.replace("-", " ")
            desc = img_caption.split()
            
            # convert lowercase
            desc = [word.lower() for word in desc]
            # remove punctuation from words
            desc = [word.translate(table) for word in desc]
            # remove articles
            desc = [word for word in desc if(len(word) > 1)]
            # remove token with numbers
            desc = [word for word in desc if(word.isalpha())]
            
            # convert to a string again
            img_caption = ''.join(desc)
            captions[img][i] = img_caption
    return captions

def text_vocabulary(descriptions):
    # build vocabulary of all unique words
    vocab = set()
    
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
        
    return vocab

def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    
dataset_images = '/tf/notebooks/captioning_image/Flicker8k_Dataset'
dataset_text = '/tf/notebooks/captioning_image/Flickr8k_text'

filename = dataset_text + '/Flickr8k.token.txt'

descriptions = all_img_captions(filename)
print('Tamanho da descrição = ', len(descriptions))

# cleaning the description
clean_descriptions = cleaning_text(descriptions)

# building vocabulary
vocabulary = text_vocabulary(clean_descriptions)
print('Tamanho do vocabulario = ', len(vocabulary))

save_descriptions(clean_descriptions, 'descriptions.txt')

Tamanho da descrição =  8092
Tamanho do vocabulario =  40091


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'